In [1]:
%load_ext autoreload
%autoreload 2

# Report: 06/19/2019

## Brief: What was done previously
Previous work discovered a new architecture that approximated a PDE

## Hypothesis
1. Including local values in feature embedding improves prediction
1. Adding valid bit-mask to kernels reduces edge effects
1. Using richer feature space reduces edge effects


## Summary of Main Results and Discussions
Modeled new architecture that uses skip layers to approximate PDE

New architecture achieves better performance fig ? with fewer model parameters. Additionally, this formulation is 
more natural for learned functions allowing approximation by traditional PDE equations after learning a deep model.


### Hypothesis 1 results and discussion
Leaned models handle noise suppression impressively well, dealing with noise with standard deviation twice as large as the range of the sample data. 

### Experiment 2: results and discussion
Put main result and conclusions here. Discuss importance/impact in terms of the project goals.


## Plan for next effort
Test architectures that remove the edge effect / dont penalize edge effects for patches
Perform hyper parameter optimization
Test training at multiple scales (LM-architecture)
Test the temporal consistency of networks
    

    

# Hypothesis 1
### Including local values in feature encoding improves prediction


In [ ]:
# Evaluate the effect of using 1x1, 3x3, and 5x5 kernels for feature encoding
import itertools
import tensorflow as tf
import src.predict_pde_recurrent
from tqdm import tqdm_notebook as tqdm
completed = [] #itertools.product([5, 3, 1], [20, 10, 5])

tf.logging.set_verbosity(tf.logging.ERROR)

for encoding_kernel_size, pred_len in itertools.product([5, 3, 1], [20, 10, 5]):
    if (encoding_kernel_size, pred_len) not in completed:
        name = 'enc_kernel_{}_test-conv_3-skip_1-cell'.format(encoding_kernel_size)
        num_batches = 15000 + pred_len * 1000        
        model = src.predict_pde_recurrent.train(net_name=name, 
                                                encoder_kernel_size=encoding_kernel_size,
                                                pred_length=pred_len, 
                                                history_length=5,
                                                num_batches=num_batches)
    



In [ ]:
# Compare accruacy of different runs
import os
import glob
# import itertools
import numpy as np
import plotly.plotly as py
import matplotlib.pyplot as plt
import plotly.graph_objs as go

completed = list(itertools.product([5, 3, 1], [20, 10, 5]))


# exp_root = 'F:\\'
exp_root = 'C:\\Users\\brandon\\source\\orbitalMechanics\\experiments\\turbulence\\pde\\'
exp_folders = [exp_root + 'enc_kernel_{}_test-conv_3-skip_1-cell_0.0005-lr_5-hist_{}-pred'.format(k, l) for (k,l) in completed]
# exp_folders = glob.glob(exp_root + '/enc_kernel_[0-9]*_test-conv_3-skip_1-cell*30-pred')
npys = ['train_accuracy_by_time.npz', 'validation_accuracy_by_time.npz']

train_data = []
validation_data = []
diff = []
print(len(exp_folders),len(completed))

for i, (directory, (k,l)) in enumerate(zip(exp_folders, completed)):
    # Load input, prediction and label
    try:
        if all([os.path.exists(os.path.join(exp_root, directory, f)) for f in npys]):
            ts = '10000'
            for key in set(train_acc.keys()).intersection(valid_acc.keys()):
                if int(key) > int(ts) or ts is None:
                    ts = key
            print(ts)
            train_acc_dir, valid_acc_dir = [os.path.join(exp_root, directory, f) for f in npys]

            with np.load(train_acc_dir) as train_acc, np.load(valid_acc_dir) as valid_acc:
                train_data.append(
                    go.Scatter(
                        x = list(range(train_acc[ts].shape[0])),
                        y = train_acc[ts], 
                        name = 'kernel_size={} len={}'.format(k, l)))
                validation_data.append(
                    go.Scatter(
                        x = list(range(valid_acc[ts].shape[0])),
                        y = valid_acc[ts], 
                        name = 'kernel_size={} len={}'.format(k, l)))
                diff.append(
                    go.Scatter(
                        x = list(range(valid_acc[ts].shape[0])),
                        y = valid_acc[ts] - train_acc[ts], 
                        name = 'kernel_size={} len={}'.format(k, l)))
        else:
            print('skipped', directory)
    except KeyError:
        continue
            
layout = go.Layout(
    title="Prediction Accuracy by Time Horizon - Train",
    xaxis=dict(
        type='linear',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Predicted time step, t',
    ),
    yaxis=dict(
        type='linear',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Mean squared validation error n = 20',
    )
)
    
fig = go.Figure(data=train_data, layout=layout)
py.iplot(fig, filename='acc_over_time_noise_study_train')

# Hypothesis 2
### Reduce scope of local values for pde estimation (lower cardinality)

In [ ]:
# Evaluate the effect of using 1x1, 3x3, and 5x5 kernels for pde layer encoding
import itertools
import tensorflow as tf
import src.predict_pde_recurrent
from tqdm import tqdm_notebook as tqdm
completed = [] #itertools.product([5, 3, 1], [20, 10, 5])

tf.logging.set_verbosity(tf.logging.ERROR)

for kernel_size, pred_len in itertools.product([5, 3, 1], [20, 10, 5]):
    if (kernel_size, pred_len) not in completed:
        name = 'kernel_size_{}-conv_3-skip_1-cell'.format(kernel_size)
        num_batches = 15000 + pred_len * 1000        
        model = src.predict_pde_recurrent.train(net_name=name, 
                                                conv_width=kernel_size,
                                                pred_length=pred_len, 
                                                history_length=5,
                                                num_batches=num_batches)

### Problem formulation
Let $x$ represent h patches of history $x = [x_{t-h}, ..., x_{t-1}]$ and y represent the target sequence $y = 
[x_{t}, x_{t+1},...,x_{t+l-1}]$.  We learn an encoder, $\phi(x) = u_0$, a decoder $\theta(u_n) = \hat{y}_n$ and a 
dynamics model $f(u_n)$ from equation \ref{eqn:ode}    
Rather 
than 
explicitly parameterizing the ODE on derivatives of $x$, we instead consider a system of ordinary differential
 equations of dimension m = 16. We then learn the encoding from 5 frames of history 
 encoding local features via
 
 Thus we learn a series of features $w_{enc}$ such that $u_0 = relu(x * w_{enc})$.
 We approximate the dynamics $f(u)$ by    $f(u_n)$ by


In [ ]:
import itertools
import src.predict_pde_recurrent
completed = [(5,5), (5,20), (5,40), (5,60)]

for (history, pred_len) in itertools.product([5, 20], [5, 20, 40, 60]):
    if (history, pred_len) not in completed:
        name = 'conv_3-skip_1-cell'
        num_batches = 15000 + pred_len * 1000
        model = src.predict_pde_recurrent.train(net_name=name, pred_length=pred_len, history_length=history, 
                                                num_batches=num_batches)

In [ ]:
# import packages 
import importlib
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import os

# Hypothesis 1

Deep recurrent networks are tolerant to sensor noise below a certain magnitude

## Measure accuracy over increasing fixed gaussian sensor noise
We add increasing magnitudes of gaussian noise to the input and predict the clean original signal. We expect, for
high levels of noise, the model to over-fit to the noise. However, after some threshold we expect the model to learn to recover from small perturbations by learning the underlying distribution.

In [ ]:
# Gaussian noise study (fixed input noise)
import numpy as np
import tensorflow as tf

from src.predict_pde_recurrent import train
from src.dataLoader.turbulence import Turbulence, RANDOM_SEED, LARGE_DATASET

# Use a fixed seed for noise    
np.random.seed(RANDOM_SEED)

for scale in [0, 0.01, 0.05, 0.1, 0.25]:
    noise_data = np.random.normal(size=(360, 279, 1000), scale=scale)
    
    loader = Turbulence(pred_length=40, dataset_idx=LARGE_DATASET, input_noise=noise_data, debug=False)
    
    train(loader=loader, dataset_idx=LARGE_DATASET, num_batches=50000, net_name='PDE_3-skip_1-cell_resnet_static_noise_{}'.format(scale))
    
    tf.reset_default_graph()


## Explore results - Hypothesis 1

In this test given a 3 layer encoder/decoder model with 250 units per layer, we see that the performance of the model is 
resistant to up to 5% noise without any degradation. Further noise causes significant increases in L2 loss porportinal to the magnitude of of noise. 

In [ ]:
# Compare accuracy of model with increasing fixed noise
import os
import plotly.plotly as py
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio
import plotly


plotly.offline.init_notebook_mode(connected=True)
plt.rc('text', usetex=True)
    
# Compare MSE vs magnitude of noise
noise = [2, 1, 0.75, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01, 0.005, 0.0025, 0.0001, 0]
noise.reverse()
train_accuracy = [1.836e-5, 1.8441e-5, 2.0871e-5, 2.0525e-5, 2.0918e-5, 2.2431e-5, 2.2850e-5, 3.6688e-5, 6.5500e-5, 7.1376e-5, 7.3004e-5, 7.1560e-5, 1.1372e-4]
validation_accuracy = [1.932e-5, 1.9536e-5, 2.1711e-5, 2.1794e-5, 2.1213e-5, 2.3923e-5, 2.4532e-5, 4.2984e-5, 1.0194e-4, 2.9481e-4, 5.4323e-4, 7.5883e-4, 2.0583e-3]

# Create a trace
trace1 = go.Scatter(
    x = noise,
    y = validation_accuracy,
    name="validation"
)
trace2 = go.Scatter(
    x = noise,
    y = train_accuracy,
    name="train"
)

data = [trace1]
layout = go.Layout(
    title="Magnitude of Sensor Noise vs L2 Loss",
    xaxis=dict(
        type='log',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Standard Deviation of Added Gaussian Noise',
    ),
    yaxis=dict(
        type='log',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Mean Squared Validation Error n=20',
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='static_noise_model')



In [ ]:
data = [trace1, trace2]
layout = go.Layout(
    title="Magnitude of Sensor Noise vs L2 Loss",
    xaxis=dict(
        type='log',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Standard Deviation of Added Gaussian Noise',
    ),
    yaxis=dict(
        type='log',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Mean Squared Error n = 20',
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='static_noise_model_comparison')

In [ ]:
# Accuracy over time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# exp_root = 'F:\\'
exp_root = 'C:\\Users\\brandon\\source\\orbitalMechanics\\experiments\\turbulence\\pde'
exp_folders = ['conv_3-skip_1-cell_velocity_and_vorticity_field_1200s.mat_0.0005-lr_h-hist_l-pred'.format(h,l) 
                for h,l in completed]
npys = ['train_accuracy_by_time.npz', 'validation_accuracy_by_time.npz']

train_data = []
validation_data = []
diff = []

for i, directory in enumerate(exp_folders):
    # Load input, prediction and label
    try:
        if all([os.path.exists(os.path.join(exp_root, directory, f)) for f in npys]):
            ts = '30000'
            train_acc_dir, valid_acc_dir = [os.path.join(exp_root, directory, f) for f in npys]

            with np.load(train_acc_dir) as train_acc, np.load(valid_acc_dir) as valid_acc:
                train_data.append(
                    go.Scatter(
                        x = list(range(train_acc[ts].shape[0])),
                        y = train_acc[ts], 
                        name = 'sigma^2={}'.format(noise[i])))
                validation_data.append(
                    go.Scatter(
                        x = list(range(valid_acc[ts].shape[0])),
                        y = valid_acc[ts], 
                        name = 'sigma^2={}'.format(noise[i])))
                diff.append(
                    go.Scatter(
                        x = list(range(valid_acc[ts].shape[0])),
                        y = valid_acc[ts] - train_acc[ts], 
                        name = 'sigma^2={}'.format(noise[i])))
        else:
            print('skipped', directory)
    except KeyError:
        continue
            
layout = go.Layout(
    title="Prediction Accuracy by Time Horizon - Train",
    xaxis=dict(
        type='linear',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Predicted time step, t',
    ),
    yaxis=dict(
        type='linear',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Mean squared validation error n = 20',
    )
)
    
fig = go.Figure(data=train_data, layout=layout)
py.iplot(fig, filename='acc_over_time_noise_study_train')



In [ ]:
layout = go.Layout(
    title="Prediction Accuracy by Time Horizon - Validation",
    xaxis=dict(
        type='linear',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Predicted time step, t',
    ),
    yaxis=dict(
        type='linear',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Mean squared validation error n = 20',
    )
)
    
fig = go.Figure(data=validation_data, layout=layout)
py.iplot(fig, filename='acc_over_time_noise_study_validation')

In [ ]:
layout = go.Layout(
    title="Prediction Accuracy by Time Horizon - Difference",
    xaxis=dict(
        type='linear',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Predicted time step, t',
    ),
    yaxis=dict(
        type='linear',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Diffrence in MSE between train and validation; n = 20',
    )
)
    
fig = go.Figure(data=diff, layout=layout)
py.iplot(fig, filename='acc_over_time_noise_study_diff')

In [ ]:
# Small magnitude of fixed noise increases generalization to new samples
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

exp_root = 'F:\\'
exp_folders = ['lstm_5_cells_dropout_60_static_noise_{}_velocity_and_vorticity_field_1200s.mat_lr0.0001'.format(n) for n in noise]
npys = ['train_accuracy_by_time.npz', 'validation_accuracy_by_time.npz']

train = []
validation = []
x = []


for i, directory in enumerate(exp_folders):
    # Load input, prediction and label
    if all([os.path.exists(os.path.join(exp_root, directory, f)) for f in npys]):
        ts = '50000'
        train_acc_dir, valid_acc_dir = [os.path.join(exp_root, directory, f) for f in npys]

        with np.load(train_acc_dir) as train_acc, np.load(valid_acc_dir) as valid_acc:
            try:
                train.append(train_acc[ts][19]) 
                validation.append(valid_acc[ts][19])
                x.append(noise[i] + 0.0001)
            except KeyError:
                continue
            
train_data = go.Scatter(
        x = x,
        y = train, 
        name = 'train',
        text=[str(p - 0.0001) for p in x])
validation_data = go.Scatter(
        x = x,
        y = validation, 
        name = 'validation',
        text=[str(p - 0.0001) for p in x])
            
layout = go.Layout(
    title="Sensor noise vs prediction error at step t+20",
    xaxis=dict(
        type='linear',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Standard deviation of added fixed gaussian noise',
    ),
    yaxis=dict(
        type='linear',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Mean squared validation error at step t+20',
    )
)
data=[validation_data]
    
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='acc_over_time_noise_study_train')


In [ ]:
# Visualize noise data
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Make sub-plots for input sequence
plt.rc('text', usetex=True)
fig, plots = plt.subplots(5, 8, figsize=(24, 17))

exp_root = 'F:\\'
exp_folders = ['lstm_5_cells_dropout_60_static_noise_{}_velocity_and_vorticity_field_1200s.mat_lr0.0001'.format(n) for n in noise]
titles = ['Sequence + N $\sig = {} \mu = 0$'.format(n) for n in noise if n != 0.0001]
titles = [r"$X + \mathcal{N}(\mu,\,\sigma^{2}=" + str(n) + ") $" for n in noise if n != 0.0001]

for j in range(5):
    for i, directory in enumerate(exp_folders[:8]):
        for file in os.listdir(os.path.join(exp_root, directory)):
            if file.endswith('.npz') and file.startswith('inputs'):
                file_path = os.path.join(exp_root, directory, file)
                with np.load(file_path) as foo:
                    try:
                        plots[j, i].imshow(np.reshape(foo['100000'][j*4, 0,:], (50,50)))
                        plots[j, i].set_title(titles[i], fontsize=16, color='gray')
                    except KeyError:
                        continue
    
for ax in fig.get_axes():
    ax.label_outer()
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
        
plt.show()

                
                



In [ ]:
# Make videos of label, prediction, and difference
import os
import cv2
import itertools
import numpy as np
import matplotlib.pyplot as plt

# Video parameters
fps = 10
height, width = 50, 50
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')

# Convert [0, 1] np.float32 to [0, 255] RGB cv2.uint8
cmap = plt.get_cmap('jet')
def convert_img(img):
    mapped = cmap(img)
    return cv2.normalize(mapped[:,:,:3], None, 255, 0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

def add_lables(img):
    font = cv2.FONT_HERSHEY_TRIPLEX 
    cv2.putText(img,'X',(10,40), font, 0.4,(0,0,0), 2, cv2.LINE_AA)
    cv2.putText(img,'f(X)',(10,90), font, 0.4,(0,0,0), 2, cv2.LINE_AA)
    cv2.putText(img,'Y',(10,140), font, 0.4,(0,0,0), 2, cv2.LINE_AA)
    cv2.putText(img,'|f(x) - y|^2', (10,190), font, 0.4,(0,0,0), 2, cv2.LINE_AA)
    cv2.putText(img,'X',(10,40), font, 0.4,(255,255,255), 1, cv2.LINE_AA)
    cv2.putText(img,'f(X)',(10,90), font, 0.4,(255,255,255), 1, cv2.LINE_AA)
    cv2.putText(img,'Y',(10,140), font, 0.4,(255,255,255), 1, cv2.LINE_AA)
    cv2.putText(img,'|f(x) - y|^2', (10,190), font, 0.4,(255,255,255), 1, cv2.LINE_AA)

# initialize video writer
video_filename = 'output.avi'

# noise = [2, 1, 0.75, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01, 0.005, 0.0025, 0.0001, 0]


exp_root = 'C:\\Users\\brandon\\source\\orbitalMechanics\\experiments\\turbulence\\pde'
exp_folders = ['kernel_size_{}-conv_3-skip_1-cell_0.0005-lr_5-hist_{}-pred'.format(h,l) 
                for h,l in itertools.product([5, 3, 1], [20, 10, 5])]
titles = ['kernel_size_{}, len={}'.format(h,l) for h,l in itertools.product([5, 3, 1], [20, 10, 5])]
npys = ['inputs.npz', 'predictions.npz', 'labels.npz']

for i, directory in enumerate(exp_folders):
    # Load input, prediction and label
    if all([os.path.exists(os.path.join(exp_root, directory, f)) for f in npys]):
        print('rendering video for', directory)
        input_dir, pred_dir, label_dir = [os.path.join(exp_root, directory, f) for f in npys]

        with np.load(input_dir) as inputs, np.load(pred_dir) as predictions, np.load(label_dir) as labels:
            ts = '10000'
            for key in set(inputs.keys()).intersection(predictions.keys()):
                if int(key) > int(ts) or ts is None:
                    ts = key
            print(ts)
            out = cv2.VideoWriter(directory + '.avi', fourcc, fps, (width*16, height*4))

            # First show input sequence
            for i in range(inputs[ts].shape[0]):
                top = np.concatenate(
                        [np.reshape(inputs[ts][i, j,:], (50,50)) for j in range(16)], axis=1)
                mid = np.concatenate(
                        [np.zeros((50, 50), dtype='float32') for _ in range(16)], axis=1)
                bot = np.concatenate(
                        [np.zeros((50, 50), dtype='float32') for _ in range(16)], axis=1)
                end = np.concatenate(
                        [np.zeros((50, 50), dtype='float32') for _ in range(16)], axis=1)
                         
                img = convert_img(np.concatenate([top, mid, bot, end], axis=0))
                add_lables(img)
                print(img.shape)
                out.write(img)
                
            # Then show prediction, label, and diffrence
            for i in range(labels[ts].shape[0]):
                top = np.concatenate(
                        [np.reshape(inputs[ts][19, j,:], (50,50)) for j in range(16)], axis=1)
                mid = np.concatenate(
                        [np.reshape(predictions[ts][i, j,:], (50,50)) for j in range(16)], axis=1)
                bot = np.concatenate(
                        [np.reshape(labels[ts][i, j,:], (50,50)) for j in range(16)], axis=1)
                end = np.concatenate(
                        [2**(np.reshape(predictions[ts][i, j,:], (50,50)) - np.reshape(labels[ts][i, j,:], (50,50))) for j in range(16)], axis=1)
                
                img = convert_img(np.concatenate([top, mid, bot, end], axis=0))
                add_lables(img)
                print(img.shape)
                out.write(img)

            # Now hold the last frame for 20 frames
            for _ in range(20):
                out.write(img)
            
            out.release()
            
    else:
        print('skipped', directory)
print('done')


In [ ]:
from IPython.core.display import display, HTML
display(HTML('<h1>Hello, world!</h1>'))

# Hypothesis 2

Deep recurrent networks are tolerant to missing samples 

# Test 1

Train modls with increasing magnitudes of random missing (zeroed) samples and compare their performance


In [ ]:
# Missing samples study (random missing pixels)
import numpy as np
import tensorflow as tf

from src.predict_turbulence_recurrent import train
from src.dataLoader.turbulence import Turbulence, RANDOM_SEED, LARGE_DATASET

# Use a fixed seed  
np.random.seed(RANDOM_SEED)

for s in [0.9, 0.75, 0.5, 0.25, 0.1]:
        
    loader = Turbulence(pred_length=20, dataset_idx=LARGE_DATASET, debug=False)
    
    train(loader=loader, dataset_idx=LARGE_DATASET, num_batches=100000, pixel_dropout=s,
          net_name='lstm_3_cells_20_pixel_dropout_{}'.format(s))
    
    tf.reset_default_graph()

# Baselines
Comparing model perfomance 

In [ ]:
# Best lstm network
import numpy as np
import tensorflow as tf

from src.predict_turbulence_recurrent import train
from src.dataLoader.turbulence import Turbulence, LARGE_DATASET

loader = Turbulence(pred_length=20, dataset_idx=LARGE_DATASET, input_noise=noise_data, debug=False)

train(loader=loader, dataset_idx=LARGE_DATASET, num_batches=100000, net_name='lstm_3_cells_+20_{}'.format(scale))

tf.reset_default_graph()

# Comparing Models

In [ ]:
# Make videos of label, prediction, and difference
import cv2

# Video parameters
fps = 8
height, width = 50, 50
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')

# Convert [0, 1] np.float32 to [0, 255] RGB cv2.uint8
cmap = plt.get_cmap('jet')
def convert_img(img):
    mapped = cmap(img)
    return cv2.normalize(mapped[:,:,:3], None, 255, 0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)


# initialize video writer
video_filename = 'output.avi'

exp_root = './experiments/turbulence/pde'
exp_folders = ['conv']
npys = ['inputs.npz', 'predictions.npz', 'labels.npz']

for i, directory in enumerate(exp_folders):
    # Load input, prediction and label
    if all([os.path.exists(os.path.join(exp_root, directory, f)) for f in npys]):
        print('done')
        ts = '10000'
        input_dir, pred_dir, label_dir = [os.path.join(exp_root, directory, f) for f in npys]

        with np.load(input_dir) as inputs, np.load(pred_dir) as predictions, np.load(label_dir) as labels:

            out = cv2.VideoWriter(video_filename, fourcc, fps, (width, height*4))

            # First show input sequence
            for i in range(inputs[ts].shape[0]):
                top = convert_img(np.reshape(inputs[ts][i, 0,:], (50,50)))
                mid = np.zeros((50, 50, 3), dtype='uint8')
                bot = np.zeros((50, 50, 3), dtype='uint8')
                dif = np.zeros((50, 50, 3), dtype='uint8')
                print(np.concatenate([top, mid, bot, dif], axis=0).shape)
                out.write(np.concatenate([top, mid, bot, dif], axis=0))
                
            # Then show prediction and label
            for i in range(labels[ts].shape[0]):
                top = convert_img(np.reshape(inputs[ts][19, 0,:], (50,50)))
                mid = convert_img(np.reshape(predictions[ts][i, 0,:], (50,50)))
                bot = convert_img(np.reshape(labels[ts][i, 0,:], (50,50)))
                dif = convert_img(10*2**(np.reshape(predictions[ts][i, 0,:], (50,50)) - np.reshape(labels[ts][i, 0,:], (50,50))))
                print(np.concatenate([top, mid, bot, dif], axis=0).shape)
                out.write(np.concatenate([top, mid, bot, dif], axis=0))
                
            # Then hold last image
            for i in range(10):
                out.write(np.concatenate([top, mid, bot, dif], axis=0))

            # Now hold the last frame and show label and prediction beneth
            out.release()
            break
print('done')



# Linear Baseline
Below we use back-propigation to learn a linear model for predicting the next frame in the sequence. Back-propigation is  overkill for linear models, howerver the traing time is so fast it's not worth loading the problem into a least-squares minimizer.

In [ ]:
import itertools
import src.predict_pde_recurrent
import tensorflow as tf
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False 
# completed = []
completed = [(5,5), (5,20)]

def lin_residual_cell(X):
    # Input 50 x 50 X history_len patch
    return tf.layers.conv2d(
        X, 1, 5, name='lin_res_layer', padding='same', activation=None)
    
    return head

def pde(X, skip_depth=3, num_features=1, kernel_size=5, encoder_kernel_size=1):
    # Input 50 x 50 X history_len patch
    # Map this patch to match the residual cell size
    head = tf.layers.conv2d(
        X, filters=num_features, kernel_size=encoder_kernel_size, activation=None, name='map_input',
        padding='valid')

    head = tf.scan(
        fn=lambda acc, _: lin_residual_cell(acc),
        elems=tf.zeros(pred_len), initializer=head, swap_memory=True)

    head = tf.map_fn(
        fn=lambda elem:
            tf.layers.conv2d(inputs=elem, filters=1, kernel_size=1, activation=None, name='map_output'),
        elems=head
    )

    head = tf.squeeze(head)

    return head

for (history, pred_len) in itertools.product([5], [5, 20, 40, 60]):
    if (history, pred_len) not in completed:
        name = 'single_relu_pde'
        num_batches = 15000 + pred_len * 1000
        model = src.predict_pde_recurrent.train(
            net_name=name, 
            pred_length=pred_len, 
            history_length=history, 
            network=pde,                                    
            num_batches=num_batches)

# Runge Kutta method

First try - just add the definition for RK4 to the network. We see that we learn more slowly (this could be bacause of
the increasing constraints on the learned model) and that for all prediction lengths greater than 5 there was basically no learning at all. We would like to try an experiment where we add prediction steps durring training but this could be complicated to implement in the current (non-eager) execution environment (tensorflow)

Because it was an issue with training - we quadruple the alloted training time and ajusted relu activations to be leaky to prevent gradients from disapearing 

In [2]:
import itertools
import src.predict_pde_recurrent
import tensorflow as tf

# from src.predict_pde_recurrent import residual_cell

completed = []
# completed = [(5,5), (5,20)]

def residual_cell(activation, skip_depth, num_features, kernel_width):
    num_filters = num_features
    kernel_size = (kernel_width, kernel_width)

    # Pass activation to last layer and cell layers
    head = activation

    # Cell layers
    for i in range(skip_depth):
        head = tf.layers.conv2d(
            head, num_filters, kernel_size, name='res_layer_{}'.format(i),
            padding='same')
        if i == skip_depth - 1:
            head += activation
        head = tf.nn.sigmoid(head)

    # Skip layer
    return head

def runge_kutta_pde(X, pred_len, is_training, skip_depth=5, num_features=8, kernel_size=5, encoder_kernel_size=1):
    # Input 50 x 50 X history_len patch
    # Map this patch to match the residual cell size
    print(X.shape)
    head = tf.layers.conv2d(
        X, filters=num_features, kernel_size=encoder_kernel_size, activation=tf.nn.sigmoid, name='map_input',
        padding='valid')
    print(head.shape)
    head = tf.layers.batch_normalization(head, training=is_training)
    def rk4(y):
        with tf.variable_scope('rk4', reuse=tf.AUTO_REUSE):
            k1 = residual_cell(y, skip_depth, num_features, kernel_size)
            k2 = residual_cell(y + 0.5 * k1, skip_depth, num_features, kernel_size)
            k3 = residual_cell(y + 0.5 * k2, skip_depth, num_features, kernel_size)
            k4 = residual_cell(y + k3, skip_depth, num_features, kernel_size)
            return (k1 + k2 + k3 + k4) / 4

    head = tf.scan(
        fn=lambda acc, _: rk4(acc),
        elems=tf.zeros(pred_len), initializer=head, swap_memory=True)
    print(head.shape)
    head = tf.map_fn(
        fn=lambda elem:
            tf.layers.conv2d(inputs=elem, filters=1, kernel_size=1, activation=tf.nn.sigmoid, name='map_output'),
        elems=head
    )
    print(head.shape)

    head = tf.squeeze(head)

    return head


In [ ]:
for (history, pred_len) in itertools.product([5], [5, 10, 20, 40]):
    if (history, pred_len) not in completed:
        name = 'runge_kutta_leaky_relu'
        num_batches = 15000 + pred_len * 4000
        model = src.predict_pde_recurrent.train(
            net_name=name, 
            pred_length=pred_len, 
            history_length=history, 
            network=runge_kutta_pde,                                    
            num_batches=num_batches)

## Using increasing prediction length durring training

In [ ]:
# Try to exclude deprecation warnings
for pred_len in range(58):
    num_batches = 6000
    model = src.predict_pde_recurrent.train(
        net_name = 'runge_kutta_growing_pred_len_sigmoid_6k_per_step_batch_norm_8_features_5_layers',
        pred_length=pred_len + 2, 
        history_length=5, 
        network=runge_kutta_pde,                                    
        num_batches=num_batches,
        starting_batch=num_batches*pred_len,
        multi_pred_len=True,
        retrain=True if pred_len > 0 else False)

W0920 12:47:47.254774  7804 deprecation_wrapper.py:119] From C:\Users\brandon\source\orbitalMechanics\src\predict_pde_recurrent.py:132: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0920 12:47:57.180961  7804 deprecation_wrapper.py:119] From C:\Users\brandon\source\orbitalMechanics\src\predict_pde_recurrent.py:151: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0920 12:47:57.180961  7804 deprecation_wrapper.py:119] From C:\Users\brandon\source\orbitalMechanics\src\predict_pde_recurrent.py:155: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0920 12:47:59.622300  7804 deprecation_wrapper.py:119] From C:\Users\brandon\source\orbitalMechanics\src\predict_pde_recurrent.py:160: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



Input shape: (?, 50, 50, 5)
Output shape: (2, ?, 50, 50)
(?, 50, 50, 5)
(?, 50, 50, 8)
(2, ?, 50, 50, 8)
(2, ?, 50, 50, 1)
Network shape: <unknown>


W0920 12:48:03.921832  7804 deprecation_wrapper.py:119] From C:\Users\brandon\source\orbitalMechanics\src\predict_pde_recurrent.py:256: The name tf.losses.mean_squared_error is deprecated. Please use tf.compat.v1.losses.mean_squared_error instead.

W0920 12:48:03.921832  7804 deprecation_wrapper.py:119] From C:\Users\brandon\source\orbitalMechanics\src\predict_pde_recurrent.py:256: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

W0920 12:48:03.955431  7804 deprecation_wrapper.py:119] From C:\Users\brandon\source\orbitalMechanics\src\predict_pde_recurrent.py:270: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0920 12:48:03.956428  7804 deprecation_wrapper.py:119] From C:\Users\brandon\source\orbitalMechanics\src\predict_pde_recurrent.py:272: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0920 12:48:04.516808  7804 deprecation_wrapper.py:119] 